In [3]:
import torch
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

квантование модели - техника уменьшения её размера и как следствие ускорения скорости инференса. Достигается это с помощью  
изменения типа тензоров весов и/или активаций. Как известно, веса нейросети это типы данных с плавающей точкой. Квантование  
же переводит их в инты. Достигается это, очевидно, домножением на некоторый скейл фактор (читай скаляр). Стратегия выбора  
такого скаляра определяет различные способоы квантования. Однако в торче есть динамическое квантование, которое использует  
не заранее заданный скаляр, а определяет его на основании значений данных, полученный в ходе рантайма. Попробуем разобраться  
с этим.

Для подготовим нейросеть, которую будем квантовать. В качестве датасета возьмем FashionMNIST

In [59]:
import torch.nn as nn 

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 6,kernel_size=5,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten()
        )

        self.fc1 = nn.Sequential(
           
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.ReLU(inplace=True),

            nn.Linear(120,84),
            nn.ReLU(inplace=True),

            nn.Linear(84,10)
        )
    
    def forward(self,x):
        out = self.cnn1(x)
        #out = out.view(out.size()[0], -1)
        out = self.fc1(out)
        return out
    

In [60]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from torch.utils.data import DataLoader

trainingData = CIFAR10(
    root = "./data/CIFAR10/",
    train = True,
    download=False,
    transform = T.ToTensor()
)
testData = CIFAR10(
    root = "./data/CIFAR10/",
    train = False,
    download = False,
    transform = T.ToTensor()
)



In [83]:
import copy
def train(model, dl, lossFunc, optimizer, device):
    size = len(dl.dataset)
    for batch, (X,y) in enumerate(dl):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X)
        loss = lossFunc(out, y)
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def validate(model, dl, lossFunc, device):
    size = len(dl.dataset)
    numBatches = len(dl)
    testLoss, correct = 0, 0
    model = model.to('cpu')
    with torch.no_grad():
        for X, y in dl:
            pred = model(X)
            testLoss += lossFunc(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    testLoss /= numBatches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")
    model = model.to(device)
    return 100*correct

def loop(model, traindl, testdl, optimizer, lossFunc, epochs, device):
    bestAcc = 0.0
    bestEp = 0
    best = None
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")
        model.train()
        train(model, traindl,lossFunc,optimizer, device)
        model.eval()
        acc = validate(model, testdl, lossFunc, device)
        if acc > bestAcc:
            bestAcc = acc
            best = copy.deepcopy(model.state_dict())
            bestEp = bestEp
    return best, bestAcc


In [70]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
trainDL = DataLoader(trainingData, shuffle=True, batch_size=64)
testDL = DataLoader(testData, shuffle=True, batch_size=64)
model = SimpleCNN()
lossF = CrossEntropyLoss()
optim = SGD(model.parameters(), lr=0.003, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
print(device)
model = model.to(device)


cuda:0


In [ ]:
loop(model, trainDL, testDL, optim, lossF, epochs, device)

In [ ]:
#torch.save(model.state_dict(), './models/best.pth')
trainDL = DataLoader(trainingData, shuffle=True, batch_size=64)
testDL = DataLoader(testData, shuffle=True, batch_size=64)

model = SimpleCNN()
model.load_state_dict(torch.load("./models/best.pth"))
model = model.to(device)
optim = torch.optim.Adam(model.parameters(), 0.0001)
bestmodel, bestacc = loop(model, trainDL, testDL, optim, lossF, epochs, device)

наверное 64% это максимум, который я получу